In [1]:
import pandas as pd
import numpy as np
import altair as alt
import time
import requests

In [17]:
data = pd.read_csv("../../data/raw/environment data.csv")

In [18]:
display(data.shape)
data.head()

(1318, 94)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,CDSD,CDSD_ATTRIBUTES,...,WSFG,WSFG_ATTRIBUTES,,,.1,.1,.2,.2,.3,Unnamed: 93
0,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1990,,,,,...,,,,,,,,,,NaN
1,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1991,,,,,...,,,,,,,,,,NaN
2,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1992,,,66,,...,,,,,,,,,,NaN
3,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1993,,,142,,...,,,,,,,,,,NaN
4,USC00437612,"SOUTH LINCOLN, VT US",44.0725,-72.9736,408.7,1994,,,144,,...,,,,,,,,,,NaN


In [19]:
data.rename(columns=lambda x: x.strip(), inplace=True)

- STATION (11 characters) is the station identification code. 
- STATION_NAME (max 50 characters) is the name of the station (usually city/airport name). This is an optional output field.
- LATITUDE (8 characters) is the latitude (decimated degrees w/Northern Hemisphere values > 0). This is an optional output field.
- LONGITUDE (9 characters) is the longitude (decimated degrees w/Western Hemisphere values < 0 and Eastern Hemisphere values > 0). This is an optional output field.
- ELEVATION (13 characters) is the elevation above mean sea level in meters (to nearest thousandth of a meter). This is an optional output field.
- DATE is the year of the record (4 digits). 


In [20]:
new_data = data.copy(deep=True)

In [21]:
lat, long = None, None
counties = []
start = time.time()
for i, j in new_data.iterrows():
    lat, long = j.loc[['LATITUDE', 'LONGITUDE']]
    payload = {'latitude': lat, 'longitude': long, 'format': 'json'}
    r = requests.get('https://geo.fcc.gov/api/census/area', params=payload).json()
    county = r['County']['name']
    counties.append(county)
print(counties)
new_data.insert(4, "COUNTY", counties, True)
new_data.head()
print(time.time() - start)

['Addison', 'Addison', 'Addison', 'Addison', 'Addison', 'Addison', 'Addison', 'Orleans', 'Orleans', 'Windham', 'Windham', 'Windham', 'Windham', 'Windham', 'Windham', 'Bennington', 'Bennington', 'Bennington', 'Bennington', 'Bennington', 'Windsor', 'Windsor', 'Windsor', 'Windsor', 'Windsor', 'Orleans', 'Orleans', 'Orleans', 'Orleans', 'Orleans', 'Orleans', 'Orange', 'Orange', 'Orange', 'Orange', 'Essex', 'Essex', 'Rutland', 'Rutland', 'Rutland', 'Rutland', 'Rutland', 'Rutland', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Washington', 'Washington', 'Grand Isle', 'Grand Isle', 'Grand Isle', 'Grand Isle', 'Grand Isle', 'Grand Isle', 'Coos', 'Coos', 'Coos', 'Coos', 'Windham', 'Windham', 'Windham', 'Windham', 'Windham', 'Chittenden', 'Chittenden', 'Chittenden', 'Chittenden', 'Chittenden', 'Chittenden', 'Washington', 'Washington', 'Washington', 'Washington', 'Washington', 'Washington', 'Franklin', 'Franklin', 'Frankli

In [22]:
new_data.to_csv("../../data/raw/environment data - with counties.csv", index=False)